In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Adjusting path
#/train_data.xml
%cd "/content/drive/MyDrive/Colab Notebooks/akmam"


/content/drive/MyDrive/Colab Notebooks/akmam


### Data Extraction from XML

In [4]:
# Loading the dataset now
import xml.etree.ElementTree as ET
tree = ET.parse('train_data.xml')
reviews = tree.getroot()


In [6]:
reviewsentencearray=[]
aspectarray=[]
polarityarray=[]
ridarray = []

i=-1

numsentences=0
for review in reviews:
  i+=1
  if(i==299):
    ridarray.append(i)
    polarityarray.append(0.0)
    aspectarray.append('FOOD#QUALITY')
    reviewsentencearray.append("Food is very bad")

  
  sentences = review.find("./sentences")
  
  for sentence in sentences:
    numsentences+=1
    
    text = sentence.find("./text")

    opinions = sentence.findall("./Opinions/Opinion")
    if(opinions == [] or opinions == None ):
      continue
    for opinion in opinions:
    
      ridarray.append(i)
      ##polarityarray.append(opinion.attrib['polarity'])
      if(opinion.attrib['polarity'] == "negative"):
        polarityarray.append(0.0)
      elif(opinion.attrib['polarity'] == "neutral"):
        polarityarray.append(0.5)
      elif(opinion.attrib['polarity'] == "positive"):
        polarityarray.append(1.0)

      aspectarray.append(opinion.attrib['category'])
      reviewsentencearray.append(text.text)


    
  
# print(numsentences)  
# print(len(reviewsentencearray))
# print(len(aspectarray))
# print(len(polarityarray))
### 2008, 2009 start of 305
print(ridarray[2008])
revIndex=[]
prev =0
i=0
for rid  in ridarray:
  if(prev == rid):
    i+=1
  else:
    revIndex.append(i)
    i=1
    prev=rid

maxi = -1
index =0
for i in range(len(revIndex)):
  if(revIndex[i] > maxi):
    maxi = revIndex[i]
    index = i
# print(revIndex[1])
print("Total number of sentences are " + str(len(reviewsentencearray)))
print("Maximim number of sentences in a review is " + str(maxi))


304
Total number of sentences are 2508
Maximim number of sentences in a review is 44


In [7]:
if( not (len(reviewsentencearray) == len(polarityarray) ==len(aspectarray))):
  print("gone wrong some where")


Till now we have extracted Info from XML, From now on let us create a dataframe out of them right now

### introducing pandas

In [8]:

#Import pandas library
import pandas as pd

  
result=[]
for j in range(len(reviewsentencearray)):

    result.append(polarityarray[j])
    
        
data =[]
# initialize list of lists
for i in range(len(reviewsentencearray)):
  data.append([ridarray[i], aspectarray[i],reviewsentencearray[i],result[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','text','polarity'])

DATA[100:150]


,rid,aspect,text,polarity
100,14,AMBIENCE#GENERAL,The ambience is pretty and nice for conversati...,1.0
101,14,RESTAURANT#MISCELLANEOUS,The ambience is pretty and nice for conversati...,1.0
102,15,LOCATION#GENERAL,If you've ever been along the river in Weehawk...,1.0
103,15,SERVICE#GENERAL,Add to that great service and great food at a ...,1.0
104,15,FOOD#QUALITY,Add to that great service and great food at a ...,1.0
105,15,FOOD#PRICES,Add to that great service and great food at a ...,1.0
106,15,FOOD#QUALITY,The lava cake dessert was incredible and I rec...,1.0
107,16,AMBIENCE#GENERAL,"Once you step into Cosette, you're miraculousl...",1.0
108,16,AMBIENCE#GENERAL,"This tiny restaurant is as cozy as it gets, wi...",1.0
109,16,FOOD#QUALITY,The food was average to above-average; the Fre...,1.0


**Done, Now let us clean the text using standard methods avaliable for US:XD**

### Cleaning Step:  Removing Links, HTML, Emoji, Punctuation

In [9]:
import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
  table = str.maketrans("", "", string.punctuation)
  return text.translate(table)



We have to remove stop words in the sentences for making it more efficient

In [10]:

import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords


stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
# calling all the functions declared above

DATA.text = DATA.text.map(lambda x: remove_URL(x))
DATA.text = DATA.text.map(lambda x: remove_html(x))
DATA.text = DATA.text.map(lambda x: remove_emoji(x))
DATA.text = DATA.text.map(lambda x: remove_punct(x))
DATA.text = DATA.text.map(remove_stopwords)
DATA

,rid,aspect,text,polarity
0,0,RESTAURANT#GENERAL,judging previous posts used good place longer,0.0
1,0,SERVICE#GENERAL,four us arrived noon place empty staff acted l...,0.0
2,0,SERVICE#GENERAL,never brought us complimentary noodles ignored...,0.0
3,0,FOOD#QUALITY,food lousy sweet salty portions tiny,0.0
4,0,FOOD#STYLE_OPTIONS,food lousy sweet salty portions tiny,0.0
...,...,...,...,...
2503,349,SERVICE#GENERAL,waitress came check us every minutes began cle...,0.0
2504,349,SERVICE#GENERAL,couldnt ignore fact reach plate one friends mi...,0.0
2505,349,SERVICE#GENERAL,put check without asking done came check bill ...,0.0
2506,349,RESTAURANT#GENERAL,wish could like place wish someone would retra...,0.0


**Recap of What we have done till now:**


1. Extracted all the relevant information from the xml file
2. Created a pandas dataframe out of the required data for making our lives simpler
3. Cleaned the data from pincuations, links, emojis, html   *italicized text*

**Now let us create word embeddings for proceeding with the implementaion**



### Tokenizing, Padding, Preparing Deliverables for embedding Matrix 

In [12]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

def create_corpus_tk(df):
    corpus = []
    for text in df["text"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
corpus = create_corpus_tk(DATA)

print("corpus[i] looks like:")
print(corpus[0])
print("---------------------------------")

maxi = -1
for i in range(len(reviewsentencearray)):
  if (len(corpus[i]) > maxi):
    maxi = len(corpus[i])

print("maximum words in a sentence are " + str(maxi))


corpus[i] looks like:
['judging', 'previous', 'posts', 'used', 'good', 'place', 'longer']
---------------------------------
maximum words in a sentence are 35


In [14]:
tokenizer = Tokenizer(num_words=1000000)
tokenizer.fit_on_texts(DATA.text)
word_index = tokenizer.word_index
print("number of unique words just for fun:", len(word_index))

number of unique words just for fun: 3052


Now let us create train sequences which can be processed with embeddings for catching the relation better

**IMPORTANT OBSERVATIONS:<br>
1) MAXIMUM NUMBER OF WORDS IN A SENTENCE IS : 35 <br>
2) MAXIMUM NUMBER OF SENTENCES IN A REVIEW IS : 44<br>
3) TOTAL NUMBER OF REVIEWS : 350**

In [16]:
import numpy as np
sentences = DATA.text
sequences = tokenizer.texts_to_sequences(DATA.text)
paddedsequences = pad_sequences(
sequences, maxlen=35, truncating="post", padding="post"
)
print(len(paddedsequences))
pp=np.array(DATA.polarity)
print(len(DATA.polarity))
print(type(paddedsequences[0]))

2508
2508
<class 'numpy.ndarray'>


In [17]:
newpaddingsequences = []
newpolarity = []
newaspect= []
newrid=[]
temp=[]
import numpy as np
padding = np.zeros((35))

j=0
for i in range(349):

  limit44 = 0
  while(ridarray[j]==i):
    newpaddingsequences.append(paddedsequences[j])
    newpolarity.append(polarityarray[j])
    newaspect.append(aspectarray[j])
    newrid.append(i)

    limit44+=1
    j+=1
    #print(j)
    #print(i)
  while(limit44 < 44):
    newpaddingsequences.append(padding)
    newpolarity.append(0.5)
    newaspect.append("FOOD#QUALITY")
    newrid.append(i)
    limit44 +=1

# print(len(newpaddingsequences))
# print(len(newpolarity[:44]))
newpaddingsequences = np.array(newpaddingsequences)
newpolarity = np.array(newpolarity)
newaspect = np.array(newaspect)
newrid = np.array(newrid)
temp = np.array(temp)
new_aspect_array=[]
for i in range(len(aspectarray)):
    new_aspect_array.append(((aspectarray[i]).split('#')))
    new_aspect_array[i][0]=new_aspect_array[i][0].lower()
    new_aspect_array[i][1]=new_aspect_array[i][1].lower()
    if(new_aspect_array[i][1]=='style_options'):
        new_aspect_array[i][1]='style'
#         print("yes")

# print((new_aspect_array))
data = []
for i in range(len(newpaddingsequences)):
  data.append([newrid[i], newaspect[i],newpaddingsequences[i],newpolarity[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','sequence','polarity'])


DATA



,rid,aspect,sequence,polarity
0,0,RESTAURANT#GENERAL,"[2058.0, 609.0, 2059.0, 843.0, 4.0, 5.0, 610.0...",0.0
1,0,SERVICE#GENERAL,"[202.0, 34.0, 516.0, 2060.0, 5.0, 411.0, 11.0,...",0.0
2,0,SERVICE#GENERAL,"[17.0, 611.0, 34.0, 612.0, 517.0, 1417.0, 2063...",0.0
3,0,FOOD#QUALITY,"[1.0, 845.0, 258.0, 613.0, 162.0, 343.0, 0.0, ...",0.0
4,0,FOOD#STYLE_OPTIONS,"[1.0, 845.0, 258.0, 613.0, 162.0, 343.0, 0.0, ...",0.0
...,...,...,...,...
15351,348,FOOD#QUALITY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.5
15352,348,FOOD#QUALITY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.5
15353,348,FOOD#QUALITY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.5
15354,348,FOOD#QUALITY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.5


This sequencing and stuff is done, now let us go to word embedddings baby

In [18]:

train_size=2009

final_train_sequences = paddedsequences[:train_size]
# print(type(final_train_sequences[4356][0]))
# final_train_aspect = newaspect[:train_size]
final_train_labels = pp[:train_size]
# for i in range(len(final_train_labels)):
#     if(final_train_labels[i]=='negative'):
#         print(i)

final_test_sequences = paddedsequences[train_size:]
# final_test_aspect = newaspect[train_size:]
final_test_labels = pp[train_size:]

print(final_test_sequences)

# train_size=300

# final_train_sequences = newpaddingsequences[:train_size*44]
# # print(type(final_train_sequences[4356][0]))
# final_train_aspect = newaspect[:train_size*44]
# final_train_labels = newpolarity[:train_size*44]
# # for i in range(len(final_train_labels)):
# #     if(final_train_labels[i]=='negative'):
# #         print(i)

# final_test_sequences = newpaddingsequences[train_size*44:]
# final_test_aspect = newaspect[train_size*44:]
# final_test_labels = newpolarity[train_size*44:]



[[  37    1    0 ...    0    0    0]
 [2821  647    0 ...    0    0    0]
 [   2    1    0 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]


### Embeddings

In [19]:
import numpy as np
embedding_dict = {}
with open("glove.twitter.27B.100d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [ ]:
embedding_dict

In [21]:
num_words = len(word_index) +1
embedding_matrix = np.zeros((num_words, 100))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [22]:
print((embedding_matrix.shape))

(3053, 100)


In [ ]:
aa=new_aspect_array[0][0]

embedding_dict.get(aa)

In [24]:
print(final_train_sequences.shape)
# print(train_sentences.shape)

(2009, 35)


In [28]:

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=35,
        trainable=False,
    )
)
model.add(Bidirectional(LSTM(15,dropout=0.5)))
model.add(Dense(1, activation="sigmoid"))



optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary

In [29]:
print(final_train_sequences.shape)

(2009, 35)


In [30]:
history = model.fit(
    final_train_sequences,
    final_train_labels,
    epochs=20,
    validation_data=(final_train_sequences,final_train_labels),
    verbose=1,
)

Epoch 1/20
63/63 [==============================] - 6s 44ms/step - loss: 0.6585 - accuracy: 0.6257 - val_loss: 0.5987 - val_accuracy: 0.7203
Epoch 2/20
63/63 [==============================] - 2s 28ms/step - loss: 0.5797 - accuracy: 0.7188 - val_loss: 0.5570 - val_accuracy: 0.7203
Epoch 3/20
63/63 [==============================] - 2s 29ms/step - loss: 0.5416 - accuracy: 0.7203 - val_loss: 0.4985 - val_accuracy: 0.7203
Epoch 4/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4911 - accuracy: 0.7317 - val_loss: 0.4503 - val_accuracy: 0.7357
Epoch 5/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4706 - accuracy: 0.7471 - val_loss: 0.4338 - val_accuracy: 0.7621
Epoch 6/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4582 - accuracy: 0.7646 - val_loss: 0.4207 - val_accuracy: 0.7894
Epoch 7/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4389 - accuracy: 0.7785 - val_loss: 0.4101 - val_accuracy: 0.7999
Epoch 8/20
63

In [31]:
print(final_test_sequences.shape,final_test_labels.shape)

(499, 35) (499,)


In [36]:
pred = model.predict(final_test_sequences)
print(pred.shape)
print(pred[0])
pred_int=[]

pred_int = pred.round().astype("int")



(499, 1)
[0.9700526]


In [37]:
# print(final_test_sequences)
print(len(final_test_labels))
ccc=0
for i in range(len(final_test_labels)):
    if(pred_int[i]==final_test_labels[i]):
#         print(pred_int[i])
        ccc+=1
print(ccc)

499
366


In [39]:
rev_polarity=[]
j=0
for j in range(len(revIndex)):
    sum_1=0
    count_review=0
    average=0
    for i in range(len(ridarray)):
        if(ridarray[i]==j):
            count_review+=1
            sum_1+=pp[i]
    average=sum_1/count_review
    average=average.round().astype("int")
    rev_polarity.append(average)

print(rev_polarity)

[0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 

In [42]:
found_polarity=[]
# print(type(pred_int[0]))
j=0
# print(len(pred))
for j in range(len(revIndex)):
    sum_1=0
    count_review=0
    average=0
    for i in range(len(pred)):

        if(ridarray[train_size+i]==j):
            count_review+=1
            sum_1+=pred_int[i]
    if(count_review==0):
        continue
    average=sum_1/count_review
    average=average.round().astype("int")
    found_polarity.append(average)



In [43]:
count=0
aa=(ridarray[train_size])
print(len(found_polarity))
for i in range(len(found_polarity)):
    if(found_polarity[i][0]==rev_polarity[aa+i]):
        count+=1
print(count)
print(count/len(found_polarity)*100,"%")

44
37
84.0909090909091 %


In [47]:
entity_matrix=[]
attribute_matrix=[]
aspect_embedding=[]

for i in range(len(new_aspect_array)):
    entity=new_aspect_array[i][0]
    entity_matrix=embedding_dict.get(entity)

    attribute=new_aspect_array[i][1]
    attribute_matrix=embedding_dict.get(attribute)

    aspect_embedding.append((attribute_matrix+entity_matrix)/2)

aspect_embedding = np.array(aspect_embedding)
print(aspect_embedding.shape)
print(len(aspect_embedding))

(2508, 100)
2508


In [48]:
akidea  = model.predict(final_train_sequences)
print(akidea.shape)
print(akidea[0])
akidea_int=[]

akidea_int = akidea.round().astype("int")



(2009, 1)
[0.24216396]


In [ ]:
step_train_sequences=[]
step_train_labels=[]
for i in range (len(akidea)):
  temp=[]
  for j in range (len(aspect_embedding[i])):

    temp.append(float(aspect_embedding[i][j]))
    
    
  
  temp.append(akidea[i])
  step_train_sequences.append(temp)
  step_train_labels.append(polarityarray[i])


import random

for i in range(len(step_train_labels)):
  if(step_train_labels[i]==0.5):
    zendaya= random.randint(0,9)
    if(zendaya < 5):
      step_train_labels[i] = 0.0
    else:
      step_train_labels[i] = 1.0

step_train_labels

In [87]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

rev_model = Sequential()
model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=101,
        trainable=False,
    )
)

#rev_model.add(Embedding(1000000, 15, input_length=101))
rev_model.add(Bidirectional(LSTM(15,dropout=0.5)))
rev_model.add(Dense(1, activation="sigmoid"))
rev_model.add(Dense(1, activation="softmax"))


optimizer = Adam(learning_rate=3e-4)

rev_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [89]:
# step_train_sequences=np.asarray(step_train_sequences).astype(np.float32)
# step_train_labels=np.asarray(step_train_labels).astype(np.float32)


history1 = rev_model.fit(
    step_train_sequences,
    step_train_labels,
    epochs=20,
    validation_data=(step_train_sequences,step_train_labels),
    verbose=1,
)

Epoch 1/20


ValueError: ignored